In [51]:
from mysql.connector import connect, Error
from omegaconf import OmegaConf
import sqlalchemy
import pandas as pd

id	channel_id	channel_name	date	text

In [14]:
# CONFIG 
conf = OmegaConf.load('../database.yaml')

In [ ]:
pd.from_di

In [6]:
# CONNECTION TO MYSQL SERVER
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [9]:
# CREATE DATABASE
db_query = "CREATE DATABASE gpb_news_external"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(db_query)
except Error as e:
    print(e)

In [13]:
# SHOW DATABASES
query = "SHOW DATABASES"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            for db in cursor:
                print(db)
except Error as e:
    print(e)

('MySQL-8300',)
('gpb_news_external',)
('information_schema',)


In [15]:
# CONNECTION TO EXISTING DATABASE
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [18]:
query = \
"""
CREATE TABLE telegram_news(
        id BIGINT,
        channel_id BIGINT,
        channel_name VARCHAR(100),
        date VARCHAR(100),
        text TEXT
)
"""
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()  # after any changes to table
except Error as e:
    print(e)

1050 (42S01): Table 'telegram_news' already exists


In [20]:
show_table_query = "DESCRIBE telegram_news"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(show_table_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print(e)

('id', b'bigint', 'YES', '', None, '')
('channel_id', b'bigint', 'YES', '', None, '')
('channel_name', b'varchar(100)', 'YES', '', None, '')
('date', b'varchar(100)', 'YES', '', None, '')
('text', b'text', 'YES', '', None, '')


In [25]:
df = pd.read_json('../output/bbcrussian.json')
df.head()

,id,channel_id,channel_name,date,text
0,39132,1003921752,bbcrussian,2022-12-19 10:46:30+00:00,Военный корабль Королевских военно-морских сил...
1,39131,1003921752,bbcrussian,2022-12-19 10:25:37+00:00,
2,39130,1003921752,bbcrussian,2022-12-19 10:25:37+00:00,
3,39129,1003921752,bbcrussian,2022-12-19 10:25:36+00:00,
4,39128,1003921752,bbcrussian,2022-12-19 10:25:36+00:00,


In [54]:
try:
    connection = sqlalchemy.create_engine(f'mysql+mysqlconnector://{conf.user}:{conf.password}@{conf.host}/{conf.database}')
    df.to_sql(con=connection, name='telegram_news', if_exists='append', index=False)
except Error as e:
    print(e)

AttributeError: 'Engine' object has no attribute 'close'

In [52]:
query = "SELECT * FROM telegram_news"
try:
    # with connect(
    #     host=conf.host,
    #     user=conf.user,
    #     password=conf.password,
    #     database=conf.database
    # ) as connection:
    connection = sqlalchemy.create_engine(f'mysql+mysqlconnector://{conf.user}:{conf.password}@{conf.host}/{conf.database}')
    query_df = pd.read_sql(
        query, 
        connection,
        # chunksize=1000
    )
    # for chunk in query_df:
        # display(chunk)
except Error as e:
    print(e)

In [53]:
query_df

,id,channel_id,channel_name,date,text
